# Pruebas de los módulos NLP

Ejecuta cada componente de `src/nlp` contra los artículos ubicados en `data_example/`, incorporando las etiquetas y clasificaciones generadas directamente sobre los archivos `.json` originales para que el vectorizador los tenga disponibles antes de la integración con el resto del sistema.


In [ ]:
from pathlib import Path
import json
from pprint import pprint

from preprocessing import detect_language

PROJECT_ROOT = Path("/home/lia/Escritorio/Proyectos/NLP/Report_Generator/Data/Data_articles")
DATA_DIR = PROJECT_ROOT / "Data_articles3"

article_paths = sorted(DATA_DIR.glob("article_*.json"))


def load_article(path: Path) -> dict:
    with open(path, encoding="utf-8") as fh:
        return json.load(fh)


def persist_article(path: Path, data: dict) -> None:
    with open(path, "w", encoding="utf-8") as fh:
        json.dump(data, fh, ensure_ascii=False, indent=2)


def update_article(path: Path, updates: dict) -> dict:
    data = load_article(path)
    data.update(updates)
    persist_article(path, data)
    return data

sample = load_article(article_paths[0])
print(f"Artículos detectados: {len(article_paths)}")
print("Idioma detectado para el primero:", detect_language(sample.get("text", "")))


Artículos detectados: 13688
Idioma detectado para el primero: es


In [2]:
from preprocessing import TextPreprocessor

preprocessor = TextPreprocessor(use_spacy=False, remove_stopwords=True)
preprocessing_results = {}

for path in article_paths:
    article = load_article(path)
    text = article.get("text", "")
    if not text:
        continue
    processed = preprocessor.preprocess_full(text)
    updates = {
        "preprocessing": {
            "cleaned": processed["cleaned"],
            "tokens": processed["tokens"],
            "token_count": processed["token_count"],
            "sentence_count": processed["sentence_count"]
        }
    }
    update_article(path, updates)
    preprocessing_results[path.name] = {
        "token_count": updates["preprocessing"]["token_count"],
        "sentence_count": updates["preprocessing"]["sentence_count"]
    }

pprint(preprocessing_results.get("article_1.json"))


{'sentence_count': 11, 'token_count': 221}


In [3]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')
from pos_analyzer import POSAnalyzer

pos_analyzer = POSAnalyzer()
pos_results = {}

for path in article_paths:
    article = load_article(path)
    text = article.get("text", "")
    if not text:
        continue
    pos_info = pos_analyzer.analyze(text)
    patterns = pos_analyzer.get_top_patterns(pos_info, n=5)
    updates = {
        "pos_analysis": {
            "tag_freq": pos_info["tag_freq"],
            "bigram_freq": pos_info["bigram_freq"],
            "trigram_freq": pos_info["trigram_freq"],
            "top_patterns": patterns
        }
    }
    update_article(path, updates)
    pos_results[path.name] = patterns

pprint(pos_results.get("article_1.json"))


[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/lia/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


{'top_bigrams': {'FW-FW': 37,
                 'JJ-NN': 29,
                 'NN-IN': 11,
                 'NN-NN': 62,
                 'NNP-NNP': 10},
 'top_tags': {'FW': 62, 'JJ': 41, 'NN': 111, 'NNP': 30, 'NNS': 19},
 'top_trigrams': {'FW-FW-FW': 23,
                  'JJ-NN-NN': 19,
                  'NN-NN-IN': 7,
                  'NN-NN-NN': 35,
                  'VBP-JJ-NN': 8}}


In [4]:
from grammar_analyzer import GrammarAnalyzer

grammar_analyzer = GrammarAnalyzer()
grammar_results = {}

for path in article_paths:
    article = load_article(path)
    text = article.get("text", "")
    if not text:
        continue
    grammar_info = grammar_analyzer.analyze(text)
    update_article(path, {"grammar_analysis": grammar_info})
    grammar_results[path.name] = {
        "chunk_counts": grammar_info["chunk_counts"],
        "top_rules": grammar_info["top_rules"][:5]
    }

pprint(grammar_results.get("article_1.json"))


{'chunk_counts': {'ADJP': 1, 'NP': 86, 'PP': 6, 'VP': 25},
 'top_rules': [{'frequency': 3,
                'rule': "PP -> ('de', 'IN') NP",
                'weight': 0.0233},
               {'frequency': 3,
                'rule': "NP -> ('bases', 'NNS')",
                'weight': 0.0233},
               {'frequency': 2,
                'rule': "NP -> ('militares', 'NNS')",
                'weight': 0.0155},
               {'frequency': 2,
                'rule': "PP -> ('en', 'IN') NP",
                'weight': 0.0155},
               {'frequency': 2,
                'rule': "NP -> ('no', 'DT') ('estuvo', 'NN')",
                'weight': 0.0155}]}


In [7]:
from regex_annotator import RegexAnnotator

regex_annotator = RegexAnnotator()
regex_results = {}

for path in article_paths:
    article = load_article(path)
    text = article.get("text", "")
    if not text:
        continue
    annotations = regex_annotator.annotate(text)
    update_article(path, {"regex_annotations": annotations})
    regex_results[path.name] = annotations["categories"]

pprint(regex_results.get("article_1.json"))
pprint(regex_results.get("article_11.json"))
pprint(regex_results.get("article_24.json"))


['CULTURA_DEPORTE_SOCIEDAD',
 'TIEMPO_RELATIVO',
 'PAISES',
 'VIOLENCIA_CRIMEN_DERECHOS_HUMANOS',
 'JURIDICO_LEGISLATIVO_CORRUPCION',
 'ACCIONES_LEGALES',
 'ANTIIMPERIALISMO_SOBERANIA',
 'ECONOMIA_COMERCIO_FINANZAS',
 'RELACIONES_INTERNACIONALES',
 'GOBIERNOS_OFICIALES',
 'ELECCIONES_PROCESOS_POLITICOS',
 'APROBACION_RECHAZO']
['CULTURA_DEPORTE_SOCIEDAD',
 'CONFLICTOS_ARMADOS_OPERACIONES',
 'LOCALIZACIONES_ESPECIFICAS',
 'INFRAESTRUCTURA_SERVICIOS',
 'PAISES',
 'GRUPOS_ESPECIFICOS',
 'ANTIIMPERIALISMO_SOBERANIA',
 'REGIONES_PROVINCIAS',
 'RELACIONES_INTERNACIONALES',
 'PROTESTAS_CONFLICTOS_SOCIALES',
 'GOBIERNOS_OFICIALES',
 'ELECCIONES_PROCESOS_POLITICOS',
 'APROBACION_RECHAZO']
['PAISES',
 'INTEGRACION_LATINOAMERICANA',
 'CIUDADES_CAPITALES',
 'REGIONES_PROVINCIAS',
 'RELACIONES_INTERNACIONALES',
 'GOBIERNOS_OFICIALES',
 'ELECCIONES_PROCESOS_POLITICOS',
 'APROBACION_RECHAZO']


In [8]:
from relation_extractor import RelationExtractor

try:
    relation_extractor = RelationExtractor()
except ValueError as exc:
    relation_extractor = None
    print(f"No se pueden extraer relaciones: {exc}")

relation_results = {}

for path in article_paths:
    article = load_article(path)
    text = article.get("text", "")
    if not text:
        continue
    if relation_extractor:
        rel_info = relation_extractor.extract(text)
    else:
        rel_info = {"entidades": [], "relaciones": [], "error": "Modelo spaCy no disponible"}
    update_article(path, {"knowledge_graph": rel_info})
    relation_results[path.name] = rel_info

pprint(relation_results.get("article_1.json"))


{'entidades': [{'nombre': 'Gobierno', 'tipo': 'Lugar'},
               {'nombre': 'Islas Galápagos', 'tipo': 'Lugar'},
               {'nombre': 'Galápagos', 'tipo': 'Lugar'},
               {'nombre': 'Movimiento Revolución Ciudadana de',
                'tipo': 'Organizacion'},
               {'nombre': 'Asamblea Nacional', 'tipo': 'Organizacion'},
               {'nombre': 'Ciudadana', 'tipo': 'Organizacion'},
               {'nombre': 'Manta', 'tipo': 'Lugar'},
               {'nombre': 'Ecuador', 'tipo': 'Lugar'}],
 'relaciones': [{'objeto': 'Asamblea Nacional',
                 'relacion': 'EMITIR',
                 'sujeto': 'Movimiento Revolución Ciudadana de'},
                {'objeto': 'Ecuador',
                 'relacion': 'EMITIR',
                 'sujeto': 'Asamblea Nacional'},
                {'objeto': 'Ciudadana',
                 'relacion': 'DENUNCIAR',
                 'sujeto': 'Asamblea Nacional'}]}


In [9]:
from sentiment_analyzer import SentimentAnalyzer

sentiment_analyzer = SentimentAnalyzer()
sentiment_results = {}

for path in article_paths:
    article = load_article(path)
    text = article.get("text", "")
    if not text:
        continue
    sentiment = sentiment_analyzer.analyze(text)
    update_article(path, {"sentiment": sentiment})
    sentiment_results[path.name] = sentiment

pprint(sentiment_results.get("article_1.json"))


{'compound': -0.9531,
 'label': 'negative',
 'neg': 0.062,
 'neu': 0.938,
 'pos': 0.0}
